# Taller Pyspark - Ciencia de datos Intermedio

In [1]:
import pandas as pd 
import numpy as np

## Escoger un dataset (que tenga tanto variables numericas como categoricas)
  - Dataset seleccionado **adult.csv** de la carpeta del curso.


## Cargar el dataset y convertirlo a archivo parquet, Json, CSV

In [2]:
df_adult = pd.read_csv('adult.csv', sep=',')

In [3]:
df_adult.to_json('adult.json', orient='records')

In [6]:
df_adult.to_parquet('adult.parquet', engine='pyarrow')

## Cargar con los diferentes formatos con Pyspark, utilizar las dos formas de caragr un dataset


In [7]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType, FloatType, IntegerType
from pyspark.sql.functions import * #  countDistinct, col, desc, asc, isnan, when, count, avg, min, max
from pyspark.sql.dataframe import DataFrame
import pyspark.sql.functions as F
spark = SparkSession.builder.appName("PySparkSession").getOrCreate()

22/11/23 19:24:07 WARN Utils: Your hostname, FACTORED-Q69X24VHF3.local resolves to a loopback address: 127.0.0.1; using 192.168.0.2 instead (on interface en0)
22/11/23 19:24:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/23 19:24:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Formas de leer CSV en Spark 

In [8]:
df_spark_format = spark.read.format("csv")\
                                .option("header", "true")\
                                .option("nullValue", "?")\
                                .option("encoding", "utf8")\
                                .load("./adult.csv")
df_spark_format.show(2)

+---+----------------+------+---------+-------------+------------------+---------------+-------------+-----+----+-----------+-----------+------------+--------------+-----+
|age|       workclass|fnlwgt|education|education-num|    marital-status|     occupation| relationship| race| sex|capitalgain|capitalloss|hoursperweek|native-country|class|
+---+----------------+------+---------+-------------+------------------+---------------+-------------+-----+----+-----------+-----------+------------+--------------+-----+
|  2|       State-gov| 77516|Bachelors|           13|     Never-married|   Adm-clerical|Not-in-family|White|Male|          1|          0|           2| United-States|<=50K|
|  3|Self-emp-not-inc| 83311|Bachelors|           13|Married-civ-spouse|Exec-managerial|      Husband|White|Male|          0|          0|           0| United-States|<=50K|
+---+----------------+------+---------+-------------+------------------+---------------+-------------+-----+----+-----------+-----------+---

In [9]:
df_spark = spark.read.csv("./adult.csv",
                            header = True,
                            sep = ",",
                            nullValue = "?",
                            encoding = "utf8")
df_spark.show(2)

+---+----------------+------+---------+-------------+------------------+---------------+-------------+-----+----+-----------+-----------+------------+--------------+-----+
|age|       workclass|fnlwgt|education|education-num|    marital-status|     occupation| relationship| race| sex|capitalgain|capitalloss|hoursperweek|native-country|class|
+---+----------------+------+---------+-------------+------------------+---------------+-------------+-----+----+-----------+-----------+------------+--------------+-----+
|  2|       State-gov| 77516|Bachelors|           13|     Never-married|   Adm-clerical|Not-in-family|White|Male|          1|          0|           2| United-States|<=50K|
|  3|Self-emp-not-inc| 83311|Bachelors|           13|Married-civ-spouse|Exec-managerial|      Husband|White|Male|          0|          0|           0| United-States|<=50K|
+---+----------------+------+---------+-------------+------------------+---------------+-------------+-----+----+-----------+-----------+---

### Formas de leer parquet con Pyspark

In [10]:
df_spark = spark.read.parquet("./adult.parquet",
                            header = True,
                            sep = ",",
                            nullValue = "?",
                            encoding = "utf8")

df_spark.show(2)

+---+----------------+------+---------+-------------+------------------+---------------+-------------+-----+----+-----------+-----------+------------+--------------+-----+
|age|       workclass|fnlwgt|education|education-num|    marital-status|     occupation| relationship| race| sex|capitalgain|capitalloss|hoursperweek|native-country|class|
+---+----------------+------+---------+-------------+------------------+---------------+-------------+-----+----+-----------+-----------+------------+--------------+-----+
|  2|       State-gov| 77516|Bachelors|           13|     Never-married|   Adm-clerical|Not-in-family|White|Male|          1|          0|           2| United-States|<=50K|
|  3|Self-emp-not-inc| 83311|Bachelors|           13|Married-civ-spouse|Exec-managerial|      Husband|White|Male|          0|          0|           0| United-States|<=50K|
+---+----------------+------+---------+-------------+------------------+---------------+-------------+-----+----+-----------+-----------+---

In [11]:
df_spark = spark.read.format("parquet")\
                        .option("header", "true")\
                        .option("nullValue", "?")\
                        .option("encoding", "utf8")\
                        .load("./adult.parquet")
df_spark.show(2)

+---+----------------+------+---------+-------------+------------------+---------------+-------------+-----+----+-----------+-----------+------------+--------------+-----+
|age|       workclass|fnlwgt|education|education-num|    marital-status|     occupation| relationship| race| sex|capitalgain|capitalloss|hoursperweek|native-country|class|
+---+----------------+------+---------+-------------+------------------+---------------+-------------+-----+----+-----------+-----------+------------+--------------+-----+
|  2|       State-gov| 77516|Bachelors|           13|     Never-married|   Adm-clerical|Not-in-family|White|Male|          1|          0|           2| United-States|<=50K|
|  3|Self-emp-not-inc| 83311|Bachelors|           13|Married-civ-spouse|Exec-managerial|      Husband|White|Male|          0|          0|           0| United-States|<=50K|
+---+----------------+------+---------+-------------+------------------+---------------+-------------+-----+----+-----------+-----------+---

### Formas de leer un Json con Pyspark

In [12]:
df_spark = spark.read.format("json")\
                        .option("encoding","utf8")\
                        .load("./adult.json")
df_spark.show(2)

+---+-----------+-----------+-----+---------+-------------+------+------------+------------------+--------------+---------------+-----+-------------+----+----------------+
|age|capitalgain|capitalloss|class|education|education-num|fnlwgt|hoursperweek|    marital-status|native-country|     occupation| race| relationship| sex|       workclass|
+---+-----------+-----------+-----+---------+-------------+------+------------+------------------+--------------+---------------+-----+-------------+----+----------------+
|  2|          1|          0|<=50K|Bachelors|           13| 77516|           2|     Never-married| United-States|   Adm-clerical|White|Not-in-family|Male|       State-gov|
|  3|          0|          0|<=50K|Bachelors|           13| 83311|           0|Married-civ-spouse| United-States|Exec-managerial|White|      Husband|Male|Self-emp-not-inc|
+---+-----------+-----------+-----+---------+-------------+------+------------+------------------+--------------+---------------+-----+-----

In [13]:
df_spark = spark.read.json("./adult.json",
                               encoding = "utf8")
df_spark.show(2)

+---+-----------+-----------+-----+---------+-------------+------+------------+------------------+--------------+---------------+-----+-------------+----+----------------+
|age|capitalgain|capitalloss|class|education|education-num|fnlwgt|hoursperweek|    marital-status|native-country|     occupation| race| relationship| sex|       workclass|
+---+-----------+-----------+-----+---------+-------------+------+------------+------------------+--------------+---------------+-----+-------------+----+----------------+
|  2|          1|          0|<=50K|Bachelors|           13| 77516|           2|     Never-married| United-States|   Adm-clerical|White|Not-in-family|Male|       State-gov|
|  3|          0|          0|<=50K|Bachelors|           13| 83311|           0|Married-civ-spouse| United-States|Exec-managerial|White|      Husband|Male|Self-emp-not-inc|
+---+-----------+-----------+-----+---------+-------------+------+------------+------------------+--------------+---------------+-----+-----

## Definir el schema de todas las columnas


In [14]:
cols_schemas = {
            "age": (LongType(), False),
            "workclass": (StringType(), True),
            "fnlwgt": (LongType(), False),
            "education": (StringType(), False),
            "education_num": (LongType(), False),
            "marital_status": (StringType(), False),
            "occupation": (StringType(), True),
            "relationship": (StringType(), False),
            "race": (StringType(), False),
            "sex": (StringType(), False),
            "capitalgain": (LongType(), False),
            "capitalloss": (LongType(), False),
            "hoursperweek": (LongType(), False),
            "native_country": (StringType(), True),
            "class": (StringType(), False)
        }

def schema(cols_schema: dict) -> StructType:
    """Define the schema of the dataframe

    Args:
        cols_schema (dict): Dictionary with the columns and its types

    Returns:
        StructType: Schema of the dataframe
    """
    return StructType([StructField(key,
                                cols_schemas[key][0],
                                cols_schemas[key][1]) for key in cols_schemas.keys()])


In [15]:
schema_df = schema(cols_schemas)

### Forma 1 para usar el schema

In [16]:
df_spark_format = spark.read.format("parquet")\
                                .option("header", "true")\
                                .option("nullValue", "?")\
                                .option("encoding", "utf8")\
                                .schema(schema_df)\
                                .load("./adult.parquet")
df_spark_format.show(2)

+---+----------------+------+---------+-------------+--------------+---------------+-------------+-----+----+-----------+-----------+------------+--------------+-----+
|age|       workclass|fnlwgt|education|education_num|marital_status|     occupation| relationship| race| sex|capitalgain|capitalloss|hoursperweek|native_country|class|
+---+----------------+------+---------+-------------+--------------+---------------+-------------+-----+----+-----------+-----------+------------+--------------+-----+
|  2|       State-gov| 77516|Bachelors|         null|          null|   Adm-clerical|Not-in-family|White|Male|          1|          0|           2|          null|<=50K|
|  3|Self-emp-not-inc| 83311|Bachelors|         null|          null|Exec-managerial|      Husband|White|Male|          0|          0|           0|          null|<=50K|
+---+----------------+------+---------+-------------+--------------+---------------+-------------+-----+----+-----------+-----------+------------+--------------

### Forma 2 para usar el schema

In [17]:
df_spark = spark.read.csv("./adult.csv",schema=schema_df,)
df_spark.show(2)

+----+---------+------+---------+-------------+--------------+------------+-------------+-----+----+-----------+-----------+------------+--------------+-----+
| age|workclass|fnlwgt|education|education_num|marital_status|  occupation| relationship| race| sex|capitalgain|capitalloss|hoursperweek|native_country|class|
+----+---------+------+---------+-------------+--------------+------------+-------------+-----+----+-----------+-----------+------------+--------------+-----+
|null|workclass|  null|education|         null|marital-status|  occupation| relationship| race| sex|       null|       null|        null|native-country|class|
|   2|State-gov| 77516|Bachelors|           13| Never-married|Adm-clerical|Not-in-family|White|Male|          1|          0|           2| United-States|<=50K|
+----+---------+------+---------+-------------+--------------+------------+-------------+-----+----+-----------+-----------+------------+--------------+-----+
only showing top 2 rows



## Calcular medidas de tendencia central (media, mediana y moda) con Pyspark y SparkSQL mínimo para la mitad de las variables


### Pyspark dataframe

In [18]:
# filter the numerical variable in the dataframe
numericas = [c for c,t in df_spark.dtypes if t in ['bigint']]
df_num = df_spark.select(numericas)
df_num.show(3)

+----+------+-------------+-----------+-----------+------------+
| age|fnlwgt|education_num|capitalgain|capitalloss|hoursperweek|
+----+------+-------------+-----------+-----------+------------+
|null|  null|         null|       null|       null|        null|
|   2| 77516|           13|          1|          0|           2|
|   3| 83311|           13|          0|          0|           0|
+----+------+-------------+-----------+-----------+------------+
only showing top 3 rows



In [19]:
def get_mean_pyspark(df_spark: DataFrame) -> dict:
        return {
            'avg_' + col: df_spark.select(
                avg(col).alias('avg_'+col)
            ).collect()[0][0]
            for col in cols_schemas.keys()
        }

In [20]:
media = get_mean_pyspark(df_spark)
media

{'avg_age': 1.7710781704270915,
 'avg_workclass': None,
 'avg_fnlwgt': 189664.13459727284,
 'avg_education': None,
 'avg_education_num': 10.078088530363212,
 'avg_marital_status': None,
 'avg_occupation': None,
 'avg_relationship': None,
 'avg_race': None,
 'avg_sex': None,
 'avg_capitalgain': 0.20031939724008027,
 'avg_capitalloss': 0.1149420580647803,
 'avg_hoursperweek': 1.9506981696081243,
 'avg_native_country': None,
 'avg_class': None}

In [21]:
def get_moda_pyspark(df_spark):
    dic_moda = {}
    list_dic = []
    for item,_ in df_spark.dtypes:
        key , value = item+'_Moda', df_spark.filter(col(item).isNotNull()).groupBy(item)\
                                                            .count()\
                                                            .orderBy("count", ascending=False)\
                                                            .collect()[0]\
                                                            .__getitem__(item)
        dic_moda[key] = value

    list_dic.append(dic_moda)
    return list_dic

In [22]:
moda = get_moda_pyspark(df_spark)
moda


[{'age_Moda': 1,
  'workclass_Moda': 'Private',
  'fnlwgt_Moda': 203488,
  'education_Moda': 'HS-grad',
  'education_num_Moda': 9,
  'marital_status_Moda': 'Married-civ-spouse',
  'occupation_Moda': 'Prof-specialty',
  'relationship_Moda': 'Husband',
  'race_Moda': 'White',
  'sex_Moda': 'Male',
  'capitalgain_Moda': 0,
  'capitalloss_Moda': 0,
  'hoursperweek_Moda': 2,
  'native_country_Moda': 'United-States',
  'class_Moda': '<=50K'}]

In [23]:
def get_median_pyspark(df_spark: DataFrame) -> dict:
    return {
        'median_' + col: df_spark.select(
            percentile_approx(col, 0.5).alias('median_'+col)
        ).collect()[0][0]
        for col in cols_schemas.keys()
    }

In [24]:
mediana = get_median_pyspark(df_spark)
mediana

{'median_age': 2,
 'median_workclass': None,
 'median_fnlwgt': 178134,
 'median_education': None,
 'median_education_num': 10,
 'median_marital_status': None,
 'median_occupation': None,
 'median_relationship': None,
 'median_race': None,
 'median_sex': None,
 'median_capitalgain': 0,
 'median_capitalloss': 0,
 'median_hoursperweek': 2,
 'median_native_country': None,
 'median_class': None}

### Pyspark SQL

In [25]:
df_spark.createOrReplaceTempView("df_spark_view")

In [26]:
avg_num_variables_string = ", ".join([f'AVG({col}) as AVG_{col}' for col,_ in df_spark.dtypes if _ in ['bigint']])

In [27]:
# get the average of the numerical variables
spark.sql(f"""SELECT
        {avg_num_variables_string}
        FROM df_spark_view""").show()

+------------------+------------------+------------------+-------------------+------------------+------------------+
|           AVG_age|        AVG_fnlwgt| AVG_education_num|    AVG_capitalgain|   AVG_capitalloss|  AVG_hoursperweek|
+------------------+------------------+------------------+-------------------+------------------+------------------+
|1.7710781704270915|189664.13459727284|10.078088530363212|0.20031939724008027|0.1149420580647803|1.9506981696081243|
+------------------+------------------+------------------+-------------------+------------------+------------------+



In [28]:
# get the mode of the categorical variables
spark.sql(f"""SELECT
        {', '.join([f'MAX({i}) as {i}_Moda' for i,_ in df_spark.dtypes])}
        FROM df_spark_view""").show()

+--------+--------------+-----------+--------------+------------------+-------------------+---------------+-----------------+---------+--------+----------------+----------------+-----------------+-------------------+----------+
|age_Moda|workclass_Moda|fnlwgt_Moda|education_Moda|education_num_Moda|marital_status_Moda|occupation_Moda|relationship_Moda|race_Moda|sex_Moda|capitalgain_Moda|capitalloss_Moda|hoursperweek_Moda|native_country_Moda|class_Moda|
+--------+--------------+-----------+--------------+------------------+-------------------+---------------+-----------------+---------+--------+----------------+----------------+-----------------+-------------------+----------+
|       4|     workclass|    1490400|     education|                16|     marital-status|     occupation|     relationship|     race|     sex|               4|               4|                4|     native-country|     class|
+--------+--------------+-----------+--------------+------------------+-----------------

In [29]:
mediana_num_variables_string = ', '.join([f"PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY {col}) OVER() AS mediana_{col}" for col,_ in df_spark.dtypes if _ in ['bigint']])

In [30]:
# get the median of the numerical variables
spark.sql(f"""SELECT
        {mediana_num_variables_string}
        FROM df_spark_view
        LIMIT 1""").show()

22/11/23 19:25:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/23 19:25:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/23 19:25:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/23 19:25:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+--------------+---------------------+-------------------+-------------------+--------------------+
|mediana_age|mediana_fnlwgt|mediana_education_num|mediana_capitalgain|mediana_capitalloss|mediana_hoursperweek|
+-----------+--------------+---------------------+-------------------+-------------------+--------------------+
|        2.0|      178144.5|                 10.0|                0.0|                0.0|                 2.0|
+-----------+--------------+---------------------+-------------------+-------------------+--------------------+



## Calcular medidas de dispersion (varianza, desviacion estandar y Coeficiente de variacion), consultar como hacerlo


### Desviación estandar

In [63]:
# calculate the dispersion measures of the numerical variables
def get_stddev_pyspark(df_spark: DataFrame) -> dict:
    return {
        'std_' + col: df_spark.select(
            stddev(col).alias('std_'+col)
        ).collect()[0][0]
        for col in cols_schemas.keys()
    }

In [76]:
get_stddev_pyspark(df_spark)

{'std_age': 1.6780046482165683,
 'std_workclass': None,
 'std_fnlwgt': 11152210185.574898,
 'std_education': None,
 'std_education_num': 6.609900909997683,
 'std_marital_status': None,
 'std_occupation': None,
 'std_relationship': None,
 'std_race': None,
 'std_sex': None,
 'std_capitalgain': 0.556787842538093,
 'std_capitalloss': 0.31016390503929436,
 'std_hoursperweek': 0.804678061309482,
 'std_native_country': None,
 'std_class': None}

### Varianza

In [ ]:
def get_variance_pyspark(df_spark: DataFrame) -> dict:
    return {
        'std_' + col: df_spark.select(
            variance(col).alias('std_'+col)
        ).collect()[0][0]
        for col in cols_schemas.keys()
    }

In [73]:
get_variance_pyspark(df_spark)

{'std_age': 1.6780046482165683,
 'std_workclass': None,
 'std_fnlwgt': 11152210185.574898,
 'std_education': None,
 'std_education_num': 6.609900909997683,
 'std_marital_status': None,
 'std_occupation': None,
 'std_relationship': None,
 'std_race': None,
 'std_sex': None,
 'std_capitalgain': 0.556787842538093,
 'std_capitalloss': 0.31016390503929436,
 'std_hoursperweek': 0.804678061309482,
 'std_native_country': None,
 'std_class': None}

### Coeficiente de variación

In [140]:
def get_coef_var_pyspark(df_spark: DataFrame) -> dict:
    variables_numericas = [c for c,t in df_spark.dtypes if t in ['bigint']]
    cv = [(F.stddev(col)/F.avg(col)).alias('cv_' + col) for col in variables_numericas]
    results = df_spark.select(cv).first()
    coef_var_dict = dict()

    for col in numericas:
        cvi= results['cv_'+ col]
        coef_var_dict[col] = cvi
    return coef_var_dict

In [141]:
get_coef_var_pyspark(df_spark)

{'age': 0.7314065572949952,
 'fnlwgt': 0.5567949135317225,
 'education_num': 0.2551051965704063,
 'capitalgain': 3.7249621760481983,
 'capitalloss': 4.845255229280509,
 'hoursperweek': 0.45985509642806494}

## Realizar mínimo 5 groupbys con Pyspark y con SparkSQL crear una tabla temporar para realizar lo mismo, y mínimo uno con pandas


### Pyspark dataframes

In [144]:
df_spark.groupBy('age').sum('education_num').show()

+----+------------------+
| age|sum(education_num)|
+----+------------------+
|   0|             90671|
|null|              null|
|   1|            130492|
|   3|             85269|
|   2|            126026|
|   4|             59776|
+----+------------------+



In [145]:
df_spark.groupBy('age','education_num').sum('hoursperweek').show()

+----+-------------+-----------------+
| age|education_num|sum(hoursperweek)|
+----+-------------+-----------------+
|   0|           13|             1765|
|   3|            5|              250|
|   2|           15|              769|
|   2|            6|              416|
|   1|            3|              201|
|   3|           14|             1710|
|   4|            7|              314|
|   0|           11|              440|
|   2|            5|              270|
|   3|           13|             2980|
|   2|           14|             2068|
|   2|            9|             7907|
|   3|           16|              472|
|   1|            1|               26|
|   0|           10|             4226|
|   0|            9|             4837|
|   3|           15|              482|
|   3|            4|              420|
|null|         null|             null|
|   4|           10|             1690|
+----+-------------+-----------------+
only showing top 20 rows



### Pyspark SQL

In [150]:
spark.sql("""
    SELECT  age, sum(education_num)
    FROM df_spark_view
    GROUP BY age
""").show()

+----+------------------+
| age|sum(education_num)|
+----+------------------+
|   0|             90671|
|null|              null|
|   1|            130492|
|   3|             85269|
|   2|            126026|
|   4|             59776|
+----+------------------+



In [149]:
spark.sql("""
    SELECT  age, education_num, sum(hoursperweek)
    FROM df_spark_view
    GROUP BY age, education_num
""").show()

+----+-------------+-----------------+
| age|education_num|sum(hoursperweek)|
+----+-------------+-----------------+
|   0|           13|             1765|
|   3|            5|              250|
|   2|           15|              769|
|   2|            6|              416|
|   1|            3|              201|
|   3|           14|             1710|
|   4|            7|              314|
|   0|           11|              440|
|   2|            5|              270|
|   3|           13|             2980|
|   2|           14|             2068|
|   2|            9|             7907|
|   3|           16|              472|
|   1|            1|               26|
|   0|           10|             4226|
|   0|            9|             4837|
|   3|           15|              482|
|   3|            4|              420|
|null|         null|             null|
|   4|           10|             1690|
+----+-------------+-----------------+
only showing top 20 rows



### Pandas

In [159]:
df_adult.groupby(['age']).agg({'education-num':'sum'})

,education-num
age,
0,90671
1,130492
2,126026
3,85269
4,59776


In [160]:
df_adult.groupby(['age','education-num']).agg({'hoursperweek':'sum'})

hoursperweek
age education-num              
0   1                        28
    2                        48
    3                       136
    4                       163
    5                       237
...                         ...
4   12                      183
    13                     1452
    14                      733
    15                      285
    16                      280

[80 rows x 1 columns]

## Buscar un dataset que incluya diferentes tablas, y con pyspark realizar un inner join, left join y right join, realizarlo tambien con SparkSQL y con Pandas


### Spark Dataframe

### Spark SQL

### Pandas

## realizar distinct count de cada columna categorica


In [177]:
categoricas = [c for c,t in df_spark.dtypes if t in ['string']]
categoricas

['workclass',
 'education',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native_country',
 'class']

In [181]:
for col in categoricas:
    df_spark.groupBy(col)\
    .agg(F.count(col)\
    .alias('Cantidad'),F.round((F.count(col)/df_spark.count())*100,2)\
    .alias('Share'))\
    .orderBy(F.col('Cantidad').desc()).show()

+----------------+--------+-----+
|       workclass|Cantidad|Share|
+----------------+--------+-----+
|         Private|   33906|69.42|
|Self-emp-not-inc|    3862| 7.91|
|       Local-gov|    3136| 6.42|
|       State-gov|    1981| 4.06|
|    Self-emp-inc|    1695| 3.47|
|     Federal-gov|    1432| 2.93|
|     Without-pay|      21| 0.04|
|    Never-worked|      10| 0.02|
|       workclass|       1|  0.0|
|            null|       0|  0.0|
+----------------+--------+-----+



+------------+--------+-----+
|   education|Cantidad|Share|
+------------+--------+-----+
|     HS-grad|   15784|32.32|
|Some-college|   10878|22.27|
|   Bachelors|    8025|16.43|
|     Masters|    2657| 5.44|
|   Assoc-voc|    2061| 4.22|
|        11th|    1812| 3.71|
|  Assoc-acdm|    1601| 3.28|
|        10th|    1389| 2.84|
|     7th-8th|     955| 1.96|
| Prof-school|     834| 1.71|
|         9th|     756| 1.55|
|        12th|     657| 1.35|
|   Doctorate|     594| 1.22|
|     5th-6th|     509| 1.04|
|     1st-4th|     247| 0.51|
|   Preschool|      83| 0.17|
|   education|       1|  0.0|
+------------+--------+-----+

+--------------------+--------+-----+
|      marital_status|Cantidad|Share|
+--------------------+--------+-----+
|  Married-civ-spouse|   22379|45.82|
|       Never-married|   16117| 33.0|
|            Divorced|    6633|13.58|
|           Separated|    1530| 3.13|
|             Widowed|    1518| 3.11|
|Married-spouse-ab...|     628| 1.29|
|   Married-AF-spouse|     

## realizar mínimo 5 conclusiones de las medias de tendencia central y de dispersion calculadas y previamente

- La edad tiene un valor muy bajo, estamos tratando con un set de adultos, se tiene la hipotesis que puede ser una llave privada o una variable categorica de la cual no tenemos un contexto adecuado.
- El 69,5% de los adultos del dataset son de workclass `Private`, seguidos por los `Self-emp-not-inc` con un 7,9%.
- dsa
- dsad
- sadsa

In [169]:
df_adult.workclass.value_counts()

Private             33906
Self-emp-not-inc     3862
Local-gov            3136
State-gov            1981
Self-emp-inc         1695
Federal-gov          1432
Without-pay            21
Never-worked           10
Name: workclass, dtype: int64